In [19]:
import os
import pandas as pd

In [20]:
folder_path = 'Forecasts'  # Replace with your folder path
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]


In [21]:
districts = ['AHMEDNAGAR', 'AKOLA', 'AMRAVATI', 'AURANGABAD', 'BEED',
       'BHANDARA', 'BULDHANA', 'CHANDRAPUR', 'DHULE', 'GADCHIROLI',
       'GONDIA', 'HINGOLI', 'JALGAON', 'JALNA', 'KOLHAPUR', 'LATUR',
       'MUMBAI', 'NAGPUR', 'NANDED', 'NANDURBAR', 'NASHIK', 'OSMANABAD',
       'PALGHAR', 'PARBHANI', 'PUNE', 'RAIGAD', 'RATNAGIRI', 'SANGLI',
       'SATARA', 'SINDHUDURG', 'SOLAPUR', 'THANE', 'WARDHA', 'WASHIM',
       'YAVATMAL', 'MUMBAI SUBURBAN']

In [22]:
dfs = []  # list to collect all df_req

for f in csv_files:
    df = pd.read_csv(f"{folder_path}/{f}")
    model = f.split('_')[0]
    
    try:
        df_req = df[['district', 'date', 'actual', 'forecast']]
    except KeyError:
        if model == 'bayesian':
            df_req = df[['district', 'date', 'actual', 'forecast_mean']]
            df_req = df_req.rename(columns={'forecast_mean': 'forecast'})  # optional: standardize column name
    df_req['model'] = model 
    dfs.append(df_req)  # collect the processed df_req

# After the loop, concatenate all collected dfs
final_df = pd.concat(dfs, ignore_index=True)

            
        
    # print(df_req.shape)

C:\Users\nauti\AppData\Local\Temp\ipykernel_40176\2311738704.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_req['model'] = model
C:\Users\nauti\AppData\Local\Temp\ipykernel_40176\2311738704.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_req['model'] = model


In [23]:
final_df.columns

Index(['district', 'date', 'actual', 'forecast', 'model'], dtype='object')

In [24]:
final_df = final_df[['model','district', 'date', 'actual', 'forecast']]

In [25]:
final_df['model'] = final_df['model'].str.upper()  # Convert district names to uppercase

In [26]:
final_df.columns

Index(['model', 'district', 'date', 'actual', 'forecast'], dtype='object')

In [27]:
import numpy as np
import pandas as pd

# Assuming 'final_df' already loaded, with columns: ['district', 'date', 'actual', 'forecast', 'model']

# Empty list to collect results
results = []

districts = final_df['district'].unique()
models = final_df['model'].unique()

for district in districts:
    for model in models:
        temp_df = final_df[(final_df['district'] == district) & (final_df['model'] == model)]
        if not temp_df.empty:
            rmse = np.sqrt(((temp_df['forecast'] - temp_df['actual'])**2).mean())
            results.append({
                'district': district,
                'model': model,
                'RMSE': rmse
            })

# Create DataFrame
rmse_df = pd.DataFrame(results)

# Sort if you want
rmse_df = rmse_df.sort_values(by=['district', 'RMSE'])

# See top results
print(rmse_df.head())


       district    model        RMSE
146  AHMEDNAGAR    LASSO  531.925517
149  AHMEDNAGAR  PROPHET  590.320304
154  AHMEDNAGAR  XGBOOST  633.931624
142  AHMEDNAGAR      EXP  938.178817
137  AHMEDNAGAR    ARIMA  944.375065


In [28]:
rmse_df[rmse_df.district == "THANE"].sort_values(by='RMSE', ascending=True)

,district,model,RMSE
600,THANE,GB,817.076035
596,THANE,DT,919.383766
595,THANE,BAYESIAN,927.378457
598,THANE,ET,992.355889
602,THANE,KNN,999.742158
604,THANE,LGBM,1097.229271
607,THANE,RF,1466.333503
610,THANE,XGBOOST,1675.490828
603,THANE,LASSO,1678.655104
609,THANE,TBATS,1811.895644


In [29]:
final_df['model'].unique()

array(['ANN', 'ARIMA', 'BAYESIAN', 'DT', 'ELASTICNET', 'ET', 'EXP', 'GB',
       'HUBER', 'KNN', 'LASSO', 'LGBM', 'LINEAR', 'LSTM', 'PROPHET', 'RF',
       'RIDGE', 'SARIMA', 'TBATS', 'XGBOOST'], dtype=object)

In [30]:
pivot_df = rmse_df.pivot(index='district', columns='model', values='RMSE')
print(pivot_df)

model                    ANN        ARIMA     BAYESIAN           DT  \
district                                                              
AHMEDNAGAR               NaN   944.375065  1045.384759  2387.388932   
AKOLA                    NaN   214.859256   184.056067   221.786606   
AMRAVATI          366.366289   410.665689   292.646152   340.073778   
AURANGABAD               NaN  1149.025138  1234.500406  1679.216440   
BEED                     NaN   701.183000   596.362735  1770.695835   
BHANDARA                 NaN   220.198093   177.472022   263.247478   
BULDHANA                 NaN   252.744098   188.372013   251.931930   
CHANDRAPUR               NaN   538.009572   501.081752   468.943464   
DHULE                    NaN   340.649834   679.649229   560.039160   
GADCHIROLI               NaN   381.429810   411.036277   556.685014   
GONDIA            177.637463   126.426029   119.867312    90.375656   
HINGOLI                  NaN    97.044835   304.969044   751.533978   
JALGAO

In [31]:
pivot_df.to_csv('I8_rmse_pivot.csv', index=True)

In [32]:
# Get the best model for each district (lowest RMSE)
best_models = rmse_df.loc[rmse_df.groupby('district')['RMSE'].idxmin()]

# Display the result
best_models[['district', 'model', 'RMSE']].to_csv('I8_best_models.csv', index=False)